# Imports

In [1]:
import json
import os
import bz2
import io
from bz2 import BZ2File
import pandas as pd
import numpy as np

# Read dataset

In [2]:
input_dir = "C:\\...\data" # create folder 'data' containing .jsonl.bz2 file and update with your path 

In [3]:
# a helper function to get the lines from am archive
def read_jsonlines(bz2_file):
    text = bz2_file.read().decode('utf-8')
    for line in text.split('\n'):
        if line != '':
            yield line

In [4]:
archives = os.listdir(input_dir)[:4] + os.listdir(input_dir)[5:]

FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'C:\\...\\data'

# Extract articles

In [ ]:
id_, journal_, date_, page_, text_ = [], [], [], [], []

for archive in archives:
    
    # take only the transformed archives
    # open the archive
    f = BZ2File(os.path.join(input_dir, archive), 'r')
        
    # get the list of articles it contains (= a json object on each line)
    articles = list(read_jsonlines(f))
    
    print(archive, ':', len(articles), 'articles à extraire')
        
    # load the first 100 articles as json and access their attributes    
    for a in articles:
         
        # decode the json string into an object (dict)
        json_article = json.loads(a)
        # print(json_article)
        if 'ft' in json_article:
            journal_.append(str(json_article["id"])[:3])
            date_.append(str(json_article["id"])[4:14])
            page_.append(str(json_article["pp"])[1:-1])
            text_.append(str(json_article["ft"]))


# Create a pandas DataFrame

In [ ]:
df = pd.DataFrame.from_dict(
    {
        'journal': journal_,
        'date': date_,
        'page': page_,
        'text': text_
    })

In [ ]:
df.head()

In [ ]:
len(df)

# Clean and enrich the DataFrame 

In [ ]:
def measure_articles(df):
    
    lengths = []
    
    for ind, row in df.iterrows():
        lengths.append(len(row['text']))
        
    return lengths

df['length'] = measure_articles(df)

In [ ]:
import re
def handle_multiple_pages(df):
    
    page, ppage = [], []
    
    for ind, row in df.iterrows():
        
        found = re.findall('([0-9]+)', row['page'])
        if len(found) > 1:
            page.append(found[0])
            ppage.append(found[1])
        else:
            page.append(row['page'])
            ppage.append(np.nan)
        
    return page, ppage
    
page, ppage = handle_multiple_pages(df)
df['page'] = page
df['ppage'] = ppage

In [ ]:
df.head()

In [ ]:
# Jeter les articles vides ou ne contenant que quelques caractères (p.ex titre des rubriques)
df = df[df['length'] > 50]

In [ ]:
# Sauvegarder l'index
df['id'] = df.index

In [ ]:
# Formater les types
df['date'] = pd.to_datetime(df['date'])
df['journal'] = df['journal'].astype('category')
df['page'] = df['page'].astype('int')
df['ppage'] = df['ppage'].astype('float')
df['text'] = df['text'].astype('str')

In [ ]:
df = df[['id', 'journal', 'date', 'page', 'ppage', 'text', 'length']]

In [ ]:
df.to_json('cleaned.json.bz2', compression = 'bz2')

In [ ]:
df_lengths = df['length'].value_counts()

In [ ]:
# Creating txt file
txt = open('data.txt', 'w+')
for ind, row in df.iterrows():
    date = str(row['date'])[0:10]
    txt.write('**** *'), txt.write(str(ind))
    txt.write(' *'), txt.write(str(row['page']))
    txt.write(' *'), txt.write(row['journal'])
    txt.write(' *'), txt.write(date), txt.write('\n')
    txt.write(row['text']), txt.write('\n')

txt.close()